# Abstractive Text Summarization

Суть задания: имеются кластера предложений. Нужно сгенерировать основное (т.е. содержащее весь смысл) предложение для каждого кластера.

$\textbf{Нужно решить, как быстрее загрузить датасет!}$

In [1]:
import pandas as pd
import numpy as np
import re

# import sberpm as pm

from rutermextract import TermExtractor
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
from nltk.tokenize import word_tokenize

from sklearn.neighbors import KNeighborsRegressor
from matplotlib import pyplot as plt
from scipy.spatial.distance import cosine
import pymorphy2

import re
# from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from corus import load_lenta

[nltk_data] Downloading package punkt to /home/ayd98/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ayd98/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/ayd98/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/ayd98/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [2]:
# pip install transformers

In [3]:
# !wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

#### load data

In [4]:
path = 'data/lenta-ru-news.csv.gz'
records = load_lenta(path)

In [5]:
df = pd.DataFrame([[rec.text, rec.title] for _, rec in zip(range(100_000), records)], columns=['text', 'title'])

In [6]:
#### preprocess

In [7]:
df['title'] = df['title'].apply(lambda x: re.sub("\xa0", " ", x))

df['text'] = df['text'].apply(lambda x: x.lower())
df['title'] = df['title'].apply(lambda x: x.lower())

In [8]:
df['tokenized_text'] = [word_tokenize(text) for text in df.text]
df['tokenized_title'] = [word_tokenize(title) for title in df.title]


In [9]:
df.to_csv('data/dataset_100k.csv')

In [10]:
df.shape

(100000, 4)

In [11]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(df['tokenized_text'].values, df['tokenized_title'].values, test_size=0.1)

#### get embeddings

In [17]:
# elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

In [ ]:
def elmo_embedding(text):
    return elmo([text])[0]

In [ ]:
train_input_embs = np.array([elmo_embedding(t) for t in train_inputs])

In [ ]:
test_input_embs = np.array([elmo_embedding(t) for t in test_inputs])

In [ ]:
train_label_embs = np.array([elmo_embedding(t) for t in train_labels])

In [ ]:
test_label_embs = np.array([elmo_embedding(t) for t in test_labels])

In [ ]:
train_ie = np.array([emb[0] for emb in train_input_embs])
test_ie = np.array([emb[0] for emb in test_input_embs])

train_le = np.array([emb[0] for emb in train_label_embs])
test_le = np.array([emb[0] for emb in test_label_embs])

#### model

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

import tensorflow.keras.backend as K

In [ ]:
model = Sequential()


model.add(Dense(1024, input_shape=(train_ie.shape), activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1024, activation='relu'))

In [ ]:
train_ie.shape[1]

1024

In [ ]:
inputs = Input(shape=(1024,))
layer1 = Dense(1024, activation='relu')(inputs)
model = Model(inputs=inputs, outputs=layer1)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.fit(train_ie, train_le, batch_size=16, epochs=10)

Train on 900 samples
Epoch 1/10
900/900 [==============================] - 0s 322us/sample - loss: -646.3714
Epoch 2/10
900/900 [==============================] - 0s 314us/sample - loss: -651.8620
Epoch 3/10
900/900 [==============================] - 0s 324us/sample - loss: -651.9393
Epoch 4/10
900/900 [==============================] - 0s 325us/sample - loss: -651.0822
Epoch 5/10
900/900 [==============================] - 0s 299us/sample - loss: -653.0007
Epoch 6/10
900/900 [==============================] - 0s 312us/sample - loss: -651.4297
Epoch 7/10
900/900 [==============================] - 0s 305us/sample - loss: -653.4815
Epoch 8/10
900/900 [==============================] - 0s 304us/sample - loss: -654.3004
Epoch 9/10
900/900 [==============================] - 0s 305us/sample - loss: -656.7020
Epoch 10/10
900/900 [==============================] - 0s 306us/sample - loss: -657.0593


In [ ]:
def my_loss:
    

ValueError: Unknown loss function:cross_entropy

In [ ]:
model(train_ie[:5])

<tf.Tensor 'sequential_4_1/dense_14/Relu:0' shape=(5, 1024) dtype=float32>

In [ ]:

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred])

NotImplementedError: Cannot convert a symbolic Tensor (sequential_2_5/dense_11/Relu:0) to a numpy array.

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 900, 1024)         1049600   
_________________________________________________________________
dense_11 (Dense)             (None, 900, 1024)         1049600   
Total params: 2,099,200
Trainable params: 2,099,200
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from transformers import BertTokenizer, TFBertModel
model_name = 'bert-base-multilingual-cased'
cache_dir = "/home/transformers_files/"
cache_dir_models = cache_dir + "bert_models/"
cache_dir_tokenizers = cache_dir + "bert_tokenizers/"
tokenizer = BertTokenizer.from_pretrained(model_name, cache_dir=cache_dir_tokenizers)
bert_model = TFBertModel.from_pretrained(model_name, cache_dir=cache_dir_models)

In [ ]:
df['tokenized_text'] = df['text'].apply(lambda x: tokenizer.encode(x, max_length=512, truncation=True))
df['tokenized_title'] = df['title'].apply(lambda x: tokenizer.encode(x, max_length=32, truncation=True))

input_ids = pad_sequences(df['tokenized_text'], maxlen=512, truncating="post", padding="post")
label_ids = pad_sequences(df['tokenized_title'], maxlen=32, truncating="post", padding="post")

In [ ]:
train_inputs, test_inputs, train_labels, test_labels = train_test_split(input_ids, label_ids, test_size=0.1)

In [ ]:
print(len(train_inputs), len(test_inputs))

**Версия с tensorflow**

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Flatten
input_layer = Input(shape=(512), dtype='int64')
bert = bert_model.bert(input_layer)
classifier = Dense(units=119547, activation='softmax')(bert[0])
model = Model(inputs=input_layer, outputs=classifier)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
bert (TFBertMainLayer)       ((None, 512, 768), (None, 177853440 
_________________________________________________________________
dense (Dense)                (None, 512, 119547)       91931643  
Total params: 269,785,083
Trainable params: 269,785,083
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
#metric = tf.keras.metrics.SparseCategoricalAccuracy()
#model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
#model.fit(train_inputs, train_labels, batch_size=4)
# было бы круто сделать loss_func from_logits=True!

In [ ]:
test_ds = tf.data.Dataset.from_tensor_slices((test_inputs, test_labels)).batch(1)
train_ds = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels)).batch(1)

In [ ]:
for inputs, labels in train_ds:
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_func(labels, predictions[:, :32, :])
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

KeyboardInterrupt: ignored

In [ ]:
import numpy as np

In [ ]:
res = np.argmax(predictions[0], axis=1)

In [ ]:
loss = loss_func(labels, predictions[:, :32, :])

In [ ]:
loss

<tf.Tensor: shape=(), dtype=float32, numpy=4.2036624>

**Версия с pytorch**

In [ ]:
import torch
test_inputs = torch.LongTensor(test_inputs)
test_labels = torch.LongTensor(test_labels)
train_inputs = torch.LongTensor(train_inputs)
train_labels = torch.LongTensor(train_labels)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
test_data = TensorDataset(test_inputs, test_labels)
train_data = TensorDataset(train_inputs, train_labels)
test_dataloader = DataLoader(test_data, batch_size=4)
train_dataloader = DataLoader(train_data, batch_size=4)

In [ ]:
from torch import nn
class FinetuneBert(nn.Module):
    def __init__(self, pretrained_bert):
        super(FinetuneBert, self).__init__()
        # dictionary_size = 119547
        # embedding_dim = 768
        self.bert = pretrained_bert
        self.ln = nn.Linear(768, 119547, bias=False)

    def forward(self, x):
        x = self.bert(x)[0]
        x = self.ln(x)
        return x

In [ ]:
model = FinetuneBert(bert_model)
for step, batch in enumerate(train_dataloader):
    b_inputs, b_labels = batch
    break
res = model(b_inputs)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

bert_model.to(device)

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
def save_checkpoint(checkpoint_path, model, optimizer):
    state = {
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    #print(f'model saved to {checkpoint_path}')
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    #print(f'model loaded from { checkpoint_path}')

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

# Будем сохранять loss во время обучения
# и рисовать график в режиме реального времени
train_loss_set = []
train_loss = 0

# Обучение, должно хватить одной эпохи
model.train()

for step, batch in enumerate(train_dataloader):

    # добавляем батч для вычисления на GPU
    batch = tuple(t.to(device) for t in batch)

    # Распаковываем данные из dataloader
    b_inputs, b_labels = batch
    
    # если не сделать .zero_grad(), градиенты будут накапливаться
    optimizer.zero_grad()

    # Forward pass
    loss = model(input_ids=b_inputs, labels=b_labels)

    train_loss_set.append(loss[0].item())  

    # Backward pass
    loss[0].backward()

    # Обновляем параметры и делаем шаг используя посчитанные градиенты
    optimizer.step()

    # Обновляем loss
    train_loss += loss[0].item()

    if loss[0].item() < min(train_loss_set):
        save_checkpoint('t5_model.pth', model, optimizer)

    # Рисуем график
    if step % 100 == 0:
        clear_output(True)
        plt.plot(train_loss_set)
        plt.title("Training loss")
        plt.xlabel("Batch")
        plt.ylabel("Loss")
        plt.show()

print("Loss на обучающей выборке: {0:.5f}".format(train_loss / len(train_dataloader)))

**Провер_очка**

In [ ]:
save_checkpoint('t5_model.pth', model, optimizer)

In [ ]:
step, batch = next(enumerate(test_dataloader))

In [ ]:
inputs, masks, labels = (t.to(device) for t in batch)

In [ ]:
res = model.generate(input_ids)

In [ ]:
text = tokenizer.decode(input_ids[2], skip_special_tokens=True)
title = tokenizer.decode(res[2], skip_special_tokens=True)